In [1]:
# Import earth engine python api and geemap 
import geemap, ee
map1 = geemap.Map()



<IPython.core.display.Javascript object>

In [16]:
# get our Ho Chi Minh, Vietnam
# I have taken level 1 data for country data from FAO datasets
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1") \
   .filter(ee.Filter.eq('ADM1_NAME',"Ho Chi Minh City")).geometry() 

In [3]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

In [4]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [5]:
# filter sentinel 2 images and apply the EVI formula, finally we obtain 
# single image with median operation

Sentinel_data = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate("2022-03-01","2022-03-31").filterBounds(aoi) \
    .map(getEVI)

In [6]:
def getLAI(image):
    LAI = image.expression(
        '(3.618*EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename("LAI")
    image = image.addBands(LAI)

    return(image)

In [7]:
Lai_image = Sentinel_data.map(getLAI).map(addDate).median().clip(aoi)

In [8]:
# set some thresholds
#  A nice EVI palette
palett = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

pall = {"min":0.5, "max":3.5, 'palette':palett}

map1.centerObject(aoi, 8)
map1.addLayer(Lai_image.select('LAI'), pall, "LAI")

map1.addLayerControl()
map1

Map(center=[10.752267385049521, 106.69138504342378], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
dataset = ee.ImageCollection('JAXA/GCOM-C/L3/LAND/LAI/V3') \
                .filterDate("2022-03-01","2022-03-31") \
                .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D')).mean().multiply(0.001).clip(aoi)

# Multiply with slope coefficient
# dataset = dataset.mean().multiply(0.001)
Map = geemap.Map()
visualization = {
  'bands': ['LAI_AVE'],
  'min': -7,
  'max': 7,
  'palette': [
    '040274','040281','0502a3','0502b8','0502ce','0502e6',
    '0602ff','235cb1','307ef3','269db1','30c8e2','32d3ef',
    '3be285','3ff38f','86e26f','3ae237','b5e22e','d6e21f',
    'fff705','ffd611','ffb613','ff8b13','ff6e08','ff500d',
    'ff0000','de0101','c21301','a71001','911003',
  ]
}

Map.setCenter(128.45, 33.33, 5)

Map.addLayer(dataset, visualization, 'Leaf Area Index')
Map

Map(center=[33.33, 128.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…